In [2]:
%load_ext autoreload

%autoreload 1

In [39]:
import random
import numpy as np
import pandas as pd
from collections import Counter
from copy import deepcopy
import sys

sys.path.append('../protosp03/recommendation/')
sys.path.append('../protosp03/data/synthetic/')
%aimport matchings
%aimport market
%aimport upskillings
%aimport make_synthetic
%aimport recommendation

In [9]:
path = "../data/taxonomy/taxonomy_V4.csv"
mastery_levels = [1, 2, 3, 4]
years = [i for i in range(2023, 2017, -1)]

In [10]:
skills, learners, jobs, courses = make_synthetic.get_job_market(path, mastery_levels, years )

In [11]:
skill_supply, skill_demand, skill_trends, skills_attractiveness = market.get_all_market_metrics(skills, mastery_levels, learners, jobs, years)

In [28]:
course_recommendation = None
learner = deepcopy(learners[0])
learner_attractiveness = market.get_learner_attractiveness(learner, years, skill_supply, skill_demand).total()
enrollable_courses = matchings.get_all_enrollable_courses(learner, courses, threshold=0.9)
print("Learner attractiveness:", learner_attractiveness)
for course in enrollable_courses:
    learner = deepcopy(learners[0])
    for skill,level in course["provided_skills"].items():
        learner["possessed_skills"][skill] = level
    tmp_attractiveness = market.get_learner_attractiveness(learner, years, skill_supply, skill_demand).total()
    if tmp_attractiveness > learner_attractiveness:
        learner_attractiveness = tmp_attractiveness
        course_recommendation = course
    print("Learner attractiveness after enrolling in course", course['provided_skills'], ":", tmp_attractiveness)

Learner attractiveness: 7.8853679389393685
Learner attractiveness after enrolling in course {'Führung in Konfliktsituationen': 4, 'Aufnahmefähigkeit': 1} : 9.455986662619317
Learner attractiveness after enrolling in course {'Mündliches Verstehen': 4, 'Organisation': 1} : 9.805409801838374
Learner attractiveness after enrolling in course {'Psychische Fertigkeit': 2} : 8.736274968417826
Learner attractiveness after enrolling in course {'Erinnerungsvermögen': 3} : 9.132533471819187
Learner attractiveness after enrolling in course {'Reaktionszeit und Schnelligkeit': 2, 'Organisation': 3} : 9.336556206454167


In [29]:
course_recommendation

{'required_skills': {'Ausdauer': 2},
 'provided_skills': {'Mündliches Verstehen': 4, 'Organisation': 1}}

In [25]:
learner_attractiveness

Counter({'Kooperieren': 0.961451247165533,
         'Führung in der strategischen Ausrichtung': 2.727891156462585,
         'Data Engineering/Data Management': 0.3419717093186481,
         'Ausdauer': 0.7184878613450042,
         'Schriftliches Verstehen': 0.23546068444027626,
         'Service Engineering': 2.9001052802073213,
         'Führung in Konfliktsituationen': 1.2851473922902497,
         'Aufnahmefähigkeit': 0.28547133138969877})

In [14]:
def up_skilling_market_advice(learner, skills, skills_attractiveness):
    """Return the up-skilling advice for a learner to increase their overall attractiveness.

    Args:
        learner (dict): Learner's profile including possessed skills and levels.
        skills_attractiveness (dict): Attractiveness of all skills for each mastery level.

    Returns:
        tuple: A tuple of the skill and level to up-skill to.
    """
    up_skilling_advice = None

    original_learner_attractiveness = sum(
        [
            skills_attractiveness[(s, l)]
            for s, l in learner["possessed_skills"].items()
        ]
    )
    learner_attractiveness = original_learner_attractiveness
    for skill, level in learner["possessed_skills"].items():
        if (skill, level+1) in skills_attractiveness:
            tmp_learner = deepcopy(learner)
            tmp_learner["possessed_skills"][skill] = level+1
            tmp_learner_attractiveness = sum(
                [
                    skills_attractiveness[(s, l)]
                    for s, l in tmp_learner["possessed_skills"].items()
                ]
            )
            if tmp_learner_attractiveness > learner_attractiveness:
                learner_attractiveness = tmp_learner_attractiveness
                up_skilling_advice = (skill, level+1)

    for skill in skills:
        if skill not in learner["possessed_skills"]:
            tmp_learner = deepcopy(learner)
            tmp_learner["possessed_skills"][skill] = 1
            tmp_learner_attractiveness = original_learner_attractiveness + skills_attractiveness[(skill, 1)] 
            if tmp_learner_attractiveness > learner_attractiveness:
                learner_attractiveness = tmp_learner_attractiveness
                up_skilling_advice = (skill, 1)

    return up_skilling_advice

In [33]:
learner = deepcopy(learners[80])
up_skilling_advice = up_skilling_market_advice(learner, skills, skills_attractiveness)
up_skilling_advice

('Management der ICT', 4)

In [30]:
enrollable_courses = matchings.get_all_enrollable_courses(learner, courses, threshold=0.5)

In [36]:
for course in enrollable_courses:
    if ('Führung in Konfliktsituationen', 4) in course["provided_skills"].items():
        print(course)

{'required_skills': {'Service Engineering': 1}, 'provided_skills': {'Führung in Konfliktsituationen': 4, 'Aufnahmefähigkeit': 1}}


In [41]:
recommendation.get_course_recommendation(learner, enrollable_courses, up_skilling_advice)

{'required_skills': {'Service Engineering': 4,
  'Zusammenarbeit in virtuellen Teams': 2},
 'provided_skills': {'Network Engineering': 3}}

In [47]:
def get_course_recommendation(learner, enrollable_courses, up_skilling_advice):
    """Get the course recommendation for a learner and a given up-skilling advice.

    Args:
        learner (dict): Learner's profile including possessed skills and levels.
        enrollable_courses (list): List of all enrollable courses.
        up_skilling_advice (tuple): A tuple of the skill and level to up-skill to.

    Returns:
        dict: The course recommendation
    """
    course_recommendation = None
    matching = 0
    for course in enrollable_courses:
        if up_skilling_advice in course["provided_skills"].items():
            print("Course", course["provided_skills"], "is recommended.")
            tmp_matching = matchings.learner_course_matching(learner, course)
            if tmp_matching > matching:
                matching = tmp_matching
                course_recommendation = course
    return course_recommendation

In [49]:
get_course_recommendation(learner, enrollable_courses, ('Network Engineering', 3))

Course {'Network Engineering': 3} is recommended.


{'required_skills': {'Service Engineering': 4,
  'Zusammenarbeit in virtuellen Teams': 2},
 'provided_skills': {'Network Engineering': 3}}